In [1]:
import ollama
import pandas as pd
from ollama import Client
from ollama import chat
import methods
import formats
import json

google_calendar = methods.get_google_calendar_service()
calendar = methods.extractCalendar(google_calendar)
print(calendar)

{'name': '545 Project Meeting', 'datetime_start': '2024-03-25T16:00:00-04:00', 'datetime_end': '2024-03-25T18:00:00-04:00', 'id': '33pmlrcvmobnp4oc9t4d64ib2o_20240325T200000Z'}
{'name': 'Dinner with my family', 'datetime_start': '2024-03-25T20:00:00-04:00', 'datetime_end': '2024-03-25T23:00:00-04:00', 'id': 'aj57e7iphbpro1st4nv9kahq3k'}
{'name': 'Dinner with friends', 'datetime_start': '2024-03-26T18:00:00-04:00', 'datetime_end': '2024-03-26T21:00:00-04:00', 'id': 'lkr7kb9rpq3q9d8tlr1q49dmc8'}
{'name': '545 Project Meeting', 'datetime_start': '2024-03-28T14:00:00-04:00', 'datetime_end': '2024-03-28T14:30:00-04:00', 'id': '4k8s6unih4r88130q6cccmv2tq_20240328T180000Z'}
{'name': 'Dearborn / Detroit Trek ', 'datetime_start': '2024-03-29T10:00:00-04:00', 'datetime_end': '2024-03-29T17:30:00-04:00', 'id': '661gjael2sgriqraj1ulo8dqre'}
{'name': '545 Project Meeting', 'datetime_start': '2024-04-01T16:00:00-04:00', 'datetime_end': '2024-04-01T18:00:00-04:00', 'id': '33pmlrcvmobnp4oc9t4d64ib2o_2

In [6]:
modelfile = '''
FROM llama2
SYSTEM You are a calendar assistant. Help me with scheduling.
'''
model = ollama.create(model='test', modelfile=modelfile)
client = Client(host='http://localhost:11434')

ConnectError: [Errno 61] Connection refused

In [ ]:
query = input()
prompt2 = "Extract intent from the following query meant for a virtual calendar assistant - " + query + ". Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."

stream = ollama.chat(
    model='llama2',
    messages=[{'role': 'user', 'content': prompt2}],
    stream=True,
)
intent = ''

for chunk in stream:
  intent += chunk['message']['content'] or ""

In [ ]:
if intent == 'moveEvent':

    response = ollama.chat(model='llama2', messages=[
    {
        'role': 'user',
        'content': query,
    },
    ])

    # might need to modify
    json_response = json.loads(response['response'])


    entities = dict(json_response)
    if not entities.get('new_time', ''):
        prompt = 'Here is my calendar for the week: ' + calendar + '\n' + ' Use it to find a new time to schedule the following event, taking into account the type of event (name)' + str(json_response) + 'Your response should only output new_time as a single string where new_time represents The time to which the event is to be moved. Use 12HR format, example output: 10:00 AM.'
        response2 = ollama.chat(
            model = 'llama2',
            messages = [{'role': 'user', 'content': prompt}],
        )
        # modify
        output = response2['response']
        print(output)
        entities['new_time'] = output
        methods.moveEvent(entities)
    else:
        methods.moveEvent(entities)

In [ ]:
if intent == 'createEvent':
    response = ollama.chat(model='llama2', messages=[
    {
        'role': 'user',
        'content': query,
    },
    ])

    # might need to modify based on response output
    json_response = json.loads(response['response'])
    print(json_response)
    entities = dict(json_response)


    if not entities.get('time', ''):
        prompt = 'Here is my calendar for the week: ' + calendar + '\n' + ' Use it to find a time to schedule the following event, taking into account the type of event (name)' + str(json_response) + 'Your response should only output new_time as a single string where new_time represents The time to which the event is to be moved. Use 12HR format, example output: 10:00 AM.'
        response2 = ollama.chat(
            model = 'llama2',
            messages = [{'role': 'user', 'content': prompt}],
        )
        # might need to modify
        output = response2['response']
        print(output)
        entities['time'] = output
        methods.createEvent(entities)
    else:
        methods.createEvent(entities)
